In [ ]:
import csv

# Sample data
data = [
    ['Name', 'Age', 'Country'],
    ['John', 30, 'USA'],
    ['Emily', 25, 'Canada'],
    ['Michael', 35, 'UK']
]

# Specify the file name
csv_file = 'example.csv'

# Writing data to CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f'CSV file "{csv_file}" has been created successfully.')


In [ ]:
from flask import Flask, redirect, url_for, render_template, request
import pandas as pd
import csv
import os
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

app = Flask(__name__)

def mlfunc(mentorcsv,entcsv,name):
    print("Inside mlfunc")
    mentors_df = pd.read_csv(mentorcsv, sep=",")
    entrepreneurs_df = pd.read_csv(entcsv, sep=",")
    entrepreneurs_df = entrepreneurs_df.tail(1)

    # Convert data into text strings
    def textify(entity):
        if 'description' in entity:
            entity_text = str(entity['description'])  # Convert to string
        else:
            entity_text = ''
        for key, value in entity.items():
            if isinstance(value, bool) and value:
                entity_text += f' {str(key).replace("needs_", "").replace("_", " ")}'  # Convert to string
        return entity_text


    mentor_texts = [textify(mentors_df.loc[mentor].to_dict()) for mentor in mentors_df.index]
    entrepreneur_texts = [textify(entrepreneurs_df.loc[entrepreneur].to_dict()) for entrepreneur in entrepreneurs_df.index]

    # Vectorize the text data using TF-IDF
    vectorizer = TfidfVectorizer()
    mentor_vectors = vectorizer.fit_transform(mentor_texts)
    entrepreneur_vectors = vectorizer.transform(entrepreneur_texts)

    # Train KNN model
    k = len(mentors_df)  # Number of neighbors to consider
    knn = NearestNeighbors(n_neighbors=k, metric='cosine')
    knn.fit(mentor_vectors)

    # Find nearest mentors for each entrepreneur with at least 75% matching rate
    matches = []
    threshold = 0.50  # Minimum cosine similarity threshold
    for i, entrepreneur_vector in enumerate(entrepreneur_vectors):
        distances, nearest_mentor_indices = knn.kneighbors(entrepreneur_vector)
        nearest_mentors = []
        for distance, idx in zip(distances[0], nearest_mentor_indices[0]):
            similarity = 1 - distance
            if similarity >= threshold:
                mentor = mentors_df.index[idx]
                mentor_expertise = mentors_df.loc[mentor].to_dict()
                entrepreneur_needs = entrepreneurs_df.loc[entrepreneurs_df.index[i]].to_dict()
                common_areas = [area for area, value in mentor_expertise.items() if value and area in entrepreneur_needs and entrepreneur_needs[area]]
                if common_areas:
                    nearest_mentors.append((mentor, similarity))
            else:
                sorted_mentors = mentors_df.sort_values(by='industry_experience_years', ascending=False)
                for mentor in sorted_mentors.index:
                    if mentor not in [m[0] for m in nearest_mentors]:
                        nearest_mentors.append((mentor, 0))
                break  # Stop iterating further since we're adding all mentors in sorted order
        matches.append(nearest_mentors)

    # Load mentor names from the mentors DataFrame
    mentor_names = mentors_df['full_name'].tolist()

    # Update the loop where matched_mentors_data is constructed to include mentor names
    matched_mentors_data = []
    for idx, entrepreneur_matches in enumerate(matches):
        entrepreneur_id = entrepreneurs_df.index[idx] + 1
        for mentor_idx, (mentor_id, similarity) in enumerate(entrepreneur_matches, start=1):
            mentor_similarity = similarity * 100 if isinstance(similarity, float) else similarity
            mentor_name = mentor_names[mentor_id]  # Get mentor name from the list
            matched_mentors_data.append({
                'Mentor': mentor_name,  # Use mentor name instead of ID
                'Similarity in %': mentor_similarity
            })

    # Create a DataFrame from the matched mentors data
    matched_mentors_df = pd.DataFrame(matched_mentors_data, index=range(1, len(matched_mentors_data) + 1))
    
    csv_file = "e_folder/"+name+".csv"

    with open(csv_file, 'w', newline='') as file:
        matched_mentors_df.to_csv(csv_file, sep=",", index=False)

    entrepreneur_names = entrepreneurs_df['full_name'].tolist()

#done(mostly)-------------------------------------------------------------------------------------------------------------------

@app.route('/')
def home():
    return render_template('Home.html')

@app.route('/contact')
def contact():
    return render_template('Contact.html')

@app.route('/about')
def about():
    return render_template('About.html')

##done -------------------------------------------------------------------------------------------------------------------------

@app.route('/submit', methods=['POST', 'GET'])
def submit():
    global person
    if request.method == 'POST':
        person = request.form.get("person")
        if "si" in request.form:
            return redirect(url_for("login"))
        elif "su" in request.form:
            return redirect(url_for("register"))
    return "error in proceeding"

#the person value should be "mentor" and "entrepreneur"-------------------------------------------------------------------------

@app.route("/register", methods=['GET','POST'])
def register():
    if request.method == 'POST':
        uname = request.form.get("one")
        myemail = request.form.get("two")
        mypass = request.form.get("three")
        user_data = {'username': uname, 'email': myemail, 'password': mypass}
        
        
        if person == "mentor":
            filename = 'm.csv'
        elif person == "entrepreneur":
            filename = 'e.csv'
        else:
            return "Error: Person not defined!"
        
        d=pd.read_csv(filename)
        d = d.append(user_data, ignore_index=True)
        d.to_csv(filename, index=False)
        
        if person=="mentor":
            return redirect(url_for('register2'))
        else:
            return redirect(url_for('register3'))
    return render_template("Register.html", p=person)

@app.route('/register2', methods=['POST', 'GET'])
def register2():
    global m_name
    filename="m_info.csv"
    if request.method == 'POST':
        m_name=request.form.get("name")
        m_industry=request.form.get("industry")
        m_indus_exp=request.form.get("industry_exp")
        m_bs=request.form.get("name2Value")
        m_ms=request.form.get("name3Value")
        m_pd=request.form.get("name4Value")
        m_oal=request.form.get("name5Value")
        m_fm=request.form.get("name6Value")
        m_hr=request.form.get("name7Value")
        m_lrc=request.form.get("name8Value")
        m_tech=request.form.get("name9Value")
        m_sg=request.form.get("name10Value")
        m_ssi=request.form.get("name11Value")
        m_textarea=request.form.get("awards")

        user_data={'full_name':m_name,'industry':m_industry,'industry_experience_years':m_indus_exp,'business_strategy':m_bs,'marketing_sales':m_ms,'product_development':m_pd,'operations_logistics':m_oal,'financial_management':m_fm,'human_resources':m_hr,'legal_regulatory_compliance':m_lrc,'technology_it':m_tech,'scaling_growth':m_sg,'sustainability_social_impact':m_ssi,'awards':m_textarea}
        d=pd.read_csv(filename)
        d = d.append(user_data, ignore_index=True)
        d.to_csv(filename, index=False)
        
        csv_file = "m_folder/"+m_name+".csv"

        with open(csv_file, 'w', newline='') as file:
            return redirect(url_for('login'))
    
    return render_template("Mentor_form.html")
        
        
@app.route('/register3', methods=['POST', 'GET'])
def register3():
    filename="e_info.csv"
    global e_name
    if request.method == 'POST':
        e_name=request.form.get("name")
        e_company=request.form.get("company")
        e_industry=request.form.get("industry")
        e_bs=request.form.get("name2Value")
        e_ms=request.form.get("name3Value")
        e_pd=request.form.get("name4Value")
        e_oal=request.form.get("name5Value")
        e_fm=request.form.get("name6Value")
        e_hr=request.form.get("name7Value")
        e_lrc=request.form.get("name8Value")
        e_tech=request.form.get("name9Value")
        e_sg=request.form.get("name10Value")
        e_ssi=request.form.get("name11Value")
        e_textarea=request.form.get("model")

        user_data={'full_name':e_name,'company_organization':e_company,'industry':e_industry,'business_strategy':e_bs,'marketing_sales':e_ms,'product_development':e_pd,'operations_logistics':e_oal,'financial_management':e_fm,'human_resources':e_hr,'legal_regulatory_compliance':e_lrc,'technology_it':e_tech,'scaling_growth':e_sg,'sustainability_social_impact':e_ssi,'description':e_textarea}
        d=pd.read_csv(filename)
        d = d.append(user_data, ignore_index=True)
        d.to_csv(filename, index=False)
        return redirect(url_for('ml'))
    
    return render_template("Entrepreneur_form.html")

@app.route("/ml", methods=['POST', 'GET'])
def ml():
    print("Inside ml route")
    if request.method == 'GET':
        print("GET request received, calling mlfunc")
        mlfunc("m_info.csv","e_info.csv",e_name)
        return redirect(url_for('login'))
    elif request.method == 'POST':
        print("POST request received, redirecting to login page")
        return redirect(url_for('login'))
    

@app.route('/login', methods=['POST', 'GET'])
def login():
    global e_name
    if request.method == 'POST':
        d={}
        uname = request.form.get("one")
        e_name=uname
        mypass = request.form.get("two")
        if person == "mentor":
            filename = 'm.csv'
        elif person == "entrepreneur":
            filename = 'e.csv'
        else:
            return None

        with open(filename, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['username'] == uname:
                    d=row
                    break;

        if uname and mypass:
            if d.get("password") == mypass:
                if person == "m":
                    return redirect(url_for('login_m'))
                else:
                    return redirect(url_for('login_e'))
            else:
                return "Login failed. Try again."
        else:
            return "Error: Missing username or password."
    else:
        return render_template("Login.html",p=person)

@app.route("/login_entrepreneur")
def login_e():
    try:
        d = pd.read_csv("e_folder/"+e_name+".csv")
        d=d.head()
        return render_template("Entrepreneur_output.html", res=d)
    except FileNotFoundError:
        return "Error: CSV file not found."

@app.route("/login_mentor")
def login_m():
    try:
        d = pd.read_csv("m_folder/"+m_name+".csv")
        #for now we show that same output
        d=d.head()
        return render_template("Entrepreneur_output.html", res=d)
    except FileNotFoundError:
        return "Error: CSV file not found."

if __name__ == "__main__":
    app.run(debug=True, port=7013, use_reloader=False)